In [36]:
neighbours = {
  'S': ['A','B'],
  'A': ['C','D','S'],
  'B': ['D','G','S'],
  'C': ['A','B','G','S'],
  'D': ['A','B'],
  'G': ['A','B','C','D']
}

S = 'S'
G = 'G'

In [37]:
neighbours = {
  'S': ['A','C'],
  'A': ['B','C','S'],
  'B': ['A','D','E'],
  'C': ['A','D','S'],
  'D': ['B','C', 'E', 'G'],
  'E': ['B', 'D', 'G'],
  'G': ['D', 'E']
}

S = 'S'
G = 'G'

In [38]:
class DoublyLinkedList:
    _items = []
    def __init__(self, items=None):
        if items is None:
            items = []
        self._items = items

    def is_empty(self):
        return len(self._items) == 0

    def length(self):
        return len(self._items)

    def head(self):
        if self.is_empty():
            return None
        return self._items[0]
    
    def tail(self):
        if self.is_empty():
            return None
        return DoublyLinkedList(self._items[1:])

    def add_to_end(self, value):
        self._items.append(value)

    def add_to_start(self, value):
        self._items.insert(0, value)

    def remove(self, value):
        self._items.remove(value)

    def __add__(self, other):
        result = DoublyLinkedList()
        result._items = self._items + other._items
        return result

    def __sub__(self, other):
        result = DoublyLinkedList()
        for item in self._items:
            if item not in other._items:
                result.add_to_end(item)
        return result

    def __contains__(self, value):
        return any(x[0] == value for x in self._items)

    def __str__(self):
        return str(self._items)

In [39]:
def MoveGen(node):
  return DoublyLinkedList(neighbours[node])

# Simple Search 1

In [40]:
def SimpleSearch1(S, G):
  open = DoublyLinkedList([S])
  while not open.is_empty():
    print(f"OPEN: {open}")
    N = open.head()
    open.remove(N)
    print(f"------------")
    if N == G:
      return N
    else:
      open = open + MoveGen(N)
  
  return None

In [41]:
SimpleSearch1(S, G)

OPEN: ['S']
------------
OPEN: ['A', 'C']
------------
OPEN: ['C', 'B', 'C', 'S']
------------
OPEN: ['B', 'C', 'S', 'A', 'D', 'S']
------------
OPEN: ['C', 'S', 'A', 'D', 'S', 'A', 'D', 'E']
------------
OPEN: ['S', 'A', 'D', 'S', 'A', 'D', 'E', 'A', 'D', 'S']
------------
OPEN: ['A', 'D', 'S', 'A', 'D', 'E', 'A', 'D', 'S', 'A', 'C']
------------
OPEN: ['D', 'S', 'A', 'D', 'E', 'A', 'D', 'S', 'A', 'C', 'B', 'C', 'S']
------------
OPEN: ['S', 'A', 'D', 'E', 'A', 'D', 'S', 'A', 'C', 'B', 'C', 'S', 'B', 'C', 'E', 'G']
------------
OPEN: ['A', 'D', 'E', 'A', 'D', 'S', 'A', 'C', 'B', 'C', 'S', 'B', 'C', 'E', 'G', 'A', 'C']
------------
OPEN: ['D', 'E', 'A', 'D', 'S', 'A', 'C', 'B', 'C', 'S', 'B', 'C', 'E', 'G', 'A', 'C', 'B', 'C', 'S']
------------
OPEN: ['E', 'A', 'D', 'S', 'A', 'C', 'B', 'C', 'S', 'B', 'C', 'E', 'G', 'A', 'C', 'B', 'C', 'S', 'B', 'C', 'E', 'G']
------------
OPEN: ['A', 'D', 'S', 'A', 'C', 'B', 'C', 'S', 'B', 'C', 'E', 'G', 'A', 'C', 'B', 'C', 'S', 'B', 'C', 'E', 'G', 'B'

'G'

# Simple Search 2

In [42]:
def SimpleSearch2(S, G):
  open = DoublyLinkedList([S])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    N = open.head()
    open.remove(N)
    closed.add_to_end(N)
    print(f"------------")
    if N == G:
      return N
    else:
      open = open + (MoveGen(N) - closed)
  
  return None

In [43]:
SimpleSearch2(S, G)

OPEN: 	['S']
CLOSED: []
------------
OPEN: 	['A', 'C']
CLOSED: ['S']
------------
OPEN: 	['C', 'B', 'C']
CLOSED: ['S', 'A']
------------
OPEN: 	['B', 'C', 'D']
CLOSED: ['S', 'A', 'C']
------------
OPEN: 	['C', 'D', 'D', 'E']
CLOSED: ['S', 'A', 'C', 'B']
------------
OPEN: 	['D', 'D', 'E', 'D']
CLOSED: ['S', 'A', 'C', 'B', 'C']
------------
OPEN: 	['D', 'E', 'D', 'E', 'G']
CLOSED: ['S', 'A', 'C', 'B', 'C', 'D']
------------
OPEN: 	['E', 'D', 'E', 'G', 'E', 'G']
CLOSED: ['S', 'A', 'C', 'B', 'C', 'D', 'D']
------------
OPEN: 	['D', 'E', 'G', 'E', 'G', 'G']
CLOSED: ['S', 'A', 'C', 'B', 'C', 'D', 'D', 'E']
------------
OPEN: 	['E', 'G', 'E', 'G', 'G', 'G']
CLOSED: ['S', 'A', 'C', 'B', 'C', 'D', 'D', 'E', 'D']
------------
OPEN: 	['G', 'E', 'G', 'G', 'G', 'G']
CLOSED: ['S', 'A', 'C', 'B', 'C', 'D', 'D', 'E', 'D', 'E']
------------


'G'

# DFS

In [44]:
def MakePairs(nodeList, parent):
  if nodeList.is_empty():
    return DoublyLinkedList([])
  else:
    return DoublyLinkedList([(nodeList.head(), parent)]) + MakePairs(nodeList.tail(), parent)

def RemoveSeen(nodeList, open, closed):
  if nodeList.is_empty():
    return DoublyLinkedList([])
  else:
    N = nodeList.head()
    if N in open or N in closed:
      return RemoveSeen(nodeList.tail(), open, closed)
    else:
      return DoublyLinkedList([N]) + RemoveSeen(nodeList.tail(), open, closed)

def SkipTo(parent, nodePairs):
  if parent == nodePairs.head()[0]:
    return nodePairs
  else:
    return SkipTo(parent, nodePairs.tail())

def ReconstructPath(nodePair, closed):
  node, parent = nodePair
  path = DoublyLinkedList([node])
  while parent is not None:
    path.add_to_start(parent)
    closed = SkipTo(parent, closed)
    _, parent = closed.head()
  return path

In [45]:
def DFS(S, G):
  open = DoublyLinkedList([(S, None)])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    nodePair = open.head()
    N, _ = nodePair
    print(f"------------")
    if N == G:
      return ReconstructPath(nodePair, closed)
    else:
      closed.add_to_start(nodePair)
      neighbours = MoveGen(N)
      newNodes = RemoveSeen(neighbours, open, closed)
      newPairs = MakePairs(newNodes, N)
      open = newPairs + open.tail()
  return DoublyLinkedList([])

print(DFS(S, G))

OPEN: 	[('S', None)]
CLOSED: []
------------
OPEN: 	[('A', 'S'), ('C', 'S')]
CLOSED: [('S', None)]
------------
OPEN: 	[('B', 'A'), ('C', 'S')]
CLOSED: [('A', 'S'), ('S', None)]
------------
OPEN: 	[('D', 'B'), ('E', 'B'), ('C', 'S')]
CLOSED: [('B', 'A'), ('A', 'S'), ('S', None)]
------------
OPEN: 	[('G', 'D'), ('E', 'B'), ('C', 'S')]
CLOSED: [('D', 'B'), ('B', 'A'), ('A', 'S'), ('S', None)]
------------
['S', 'A', 'B', 'D', 'G']


# BFS

In [46]:
def BFS(S, G):
  open = DoublyLinkedList([(S, None)])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    nodePair = open.head()
    N, _ = nodePair
    print(f"------------")
    if N == G:
      return ReconstructPath(nodePair, closed)
    else:
      closed.add_to_start(nodePair)
      neighbours = MoveGen(N)
      newNodes = RemoveSeen(neighbours, open, closed)
      newPairs = MakePairs(newNodes, N)
      open = open.tail() + newPairs
  return DoublyLinkedList([])

print(BFS(S, G))

OPEN: 	[('S', None)]
CLOSED: []
------------
OPEN: 	[('A', 'S'), ('C', 'S')]
CLOSED: [('S', None)]
------------
OPEN: 	[('C', 'S'), ('B', 'A')]
CLOSED: [('A', 'S'), ('S', None)]
------------
OPEN: 	[('B', 'A'), ('D', 'C')]
CLOSED: [('C', 'S'), ('A', 'S'), ('S', None)]
------------
OPEN: 	[('D', 'C'), ('E', 'B')]
CLOSED: [('B', 'A'), ('C', 'S'), ('A', 'S'), ('S', None)]
------------
OPEN: 	[('E', 'B'), ('G', 'D')]
CLOSED: [('D', 'C'), ('B', 'A'), ('C', 'S'), ('A', 'S'), ('S', None)]
------------
OPEN: 	[('G', 'D')]
CLOSED: [('E', 'B'), ('D', 'C'), ('B', 'A'), ('C', 'S'), ('A', 'S'), ('S', None)]
------------
['S', 'C', 'D', 'G']


# DFID-N

In [47]:
def MakePairs(nodeList, parent, depth):
  if nodeList.is_empty():
    return DoublyLinkedList([])
  else:
    return DoublyLinkedList([(nodeList.head(), parent, depth)]) + MakePairs(nodeList.tail(), parent, depth)

def RemoveSeen(nodeList, open, closed):
  if nodeList.is_empty():
    return DoublyLinkedList([])
  else:
    N = nodeList.head()
    if N in open or N in closed:
      return RemoveSeen(nodeList.tail(), open, closed)
    else:
      return DoublyLinkedList([N]) + RemoveSeen(nodeList.tail(), open, closed)

def SkipTo(parent, nodePairs, depth):
  if parent == nodePairs.head()[0] and depth == nodePairs.head()[2]:
    return nodePairs
  else:
    return SkipTo(parent, nodePairs.tail(), depth)

def ReconstructPath(nodePair, closed):
  node, parent, depth = nodePair
  path = DoublyLinkedList([node])
  while parent is not None:
    path.add_to_start(parent)
    closed = SkipTo(parent, closed, depth - 1)
    _, parent, depth = closed.head()
  return path

In [48]:
def DB_DFS_N(S, G, depth_bound):
  count = 0
  open = DoublyLinkedList([(S, None, 0)])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    nodePair = open.head()
    N, _, depth = nodePair
    
    if N == G:
      return (count, ReconstructPath(nodePair, closed))
    else:
      print(f"------------")
      closed.add_to_start(nodePair)

      if depth < depth_bound:
        neighbours = MoveGen(N)
        newNodes = RemoveSeen(neighbours, open, closed)
        newPairs = MakePairs(newNodes, N, depth+1)
        open = newPairs + open.tail()
        count = count + newPairs.length()
      else:
        open = open.tail()
  return (count, DoublyLinkedList([]))

def DFID_N(S, G):
  count = -1
  path = DoublyLinkedList([])
  depth_bound = 0

  while True:
    previous_count = count
    print(f"\n==========Depth {depth_bound}============")
    count, path = DB_DFS_N(S, G, depth_bound)
    depth_bound += 1
    if previous_count == count or not path.is_empty():
      print("\n=======================")
      break

  return path

print(DFID_N(S, G))


==========Depth 0============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------

==========Depth 1============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------
OPEN: 	[('A', 'S', 1), ('C', 'S', 1)]
CLOSED: [('S', None, 0)]
------------
OPEN: 	[('C', 'S', 1)]
CLOSED: [('A', 'S', 1), ('S', None, 0)]
------------

==========Depth 2============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------
OPEN: 	[('A', 'S', 1), ('C', 'S', 1)]
CLOSED: [('S', None, 0)]
------------
OPEN: 	[('B', 'A', 2), ('C', 'S', 1)]
CLOSED: [('A', 'S', 1), ('S', None, 0)]
------------
OPEN: 	[('C', 'S', 1)]
CLOSED: [('B', 'A', 2), ('A', 'S', 1), ('S', None, 0)]
------------
OPEN: 	[('D', 'C', 2)]
CLOSED: [('C', 'S', 1), ('B', 'A', 2), ('A', 'S', 1), ('S', None, 0)]
------------

==========Depth 3============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------
OPEN: 	[('A', 'S', 1), ('C', 'S', 1)]
CLOSED: [('S', None, 0)]
------------
OPEN: 	[('B', 'A', 2), ('C', 'S', 1)]
CLOSED: [('A', 'S', 1), ('S', None, 0)]
------------
O

# DFID-C

In [49]:
def DB_DFS_C(S, G, depth_bound):
  count = 0
  open = DoublyLinkedList([(S, None, 0)])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    nodePair = open.head()
    N, _, depth = nodePair
    
    if N == G:
      return (count, ReconstructPath(nodePair, closed))
    else:
      print(f"------------")
      closed.add_to_start(nodePair)

      if depth < depth_bound:
        neighbours = MoveGen(N)
        newNodes = RemoveSeen(neighbours, open, DoublyLinkedList([]))
        newPairs = MakePairs(newNodes, N, depth+1)
        open = newPairs + open.tail()
        count = count + newPairs.length()
      else:
        open = open.tail()
  return (count, DoublyLinkedList([]))

def DFID_C(S, G):
  count = -1
  path = DoublyLinkedList([])
  depth_bound = 0

  while True:
    previous_count = count
    print(f"==========Depth {depth_bound}============")
    count, path = DB_DFS_C(S, G, depth_bound)
    depth_bound += 1
    if previous_count == count or not path.is_empty():
      print("\n=======================")
      break

  return path

print(DFID_C(S, G))

==========Depth 0============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------
==========Depth 1============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------
OPEN: 	[('A', 'S', 1), ('C', 'S', 1)]
CLOSED: [('S', None, 0)]
------------
OPEN: 	[('C', 'S', 1)]
CLOSED: [('A', 'S', 1), ('S', None, 0)]
------------
==========Depth 2============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------
OPEN: 	[('A', 'S', 1), ('C', 'S', 1)]
CLOSED: [('S', None, 0)]
------------
OPEN: 	[('B', 'A', 2), ('S', 'A', 2), ('C', 'S', 1)]
CLOSED: [('A', 'S', 1), ('S', None, 0)]
------------
OPEN: 	[('S', 'A', 2), ('C', 'S', 1)]
CLOSED: [('B', 'A', 2), ('A', 'S', 1), ('S', None, 0)]
------------
OPEN: 	[('C', 'S', 1)]
CLOSED: [('S', 'A', 2), ('B', 'A', 2), ('A', 'S', 1), ('S', None, 0)]
------------
OPEN: 	[('A', 'C', 2), ('D', 'C', 2), ('S', 'C', 2)]
CLOSED: [('C', 'S', 1), ('S', 'A', 2), ('B', 'A', 2), ('A', 'S', 1), ('S', None, 0)]
------------
OPEN: 	[('D', 'C', 2), ('S', 'C', 2)]
CLOSED: [('A', 'C', 2), ('C', '